In [ ]:
import  pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re 
import pyautogui
import warnings
warnings.filterwarnings("ignore")





keyword = input("insert keyword, 검색어를 입력하세요.")

# 뉴스의 url로 부터 응답요청
#query의 값으로 사용자로부터 입력받은 keyword가 대입
response = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+keyword)










# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["검색어", "순번", "한자","품사","설명","링크"]
df = pd.DataFrame(columns=columns)
excutable_path = "../chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)
base_url = 'https://stdict.korean.go.kr'

def urlmake(links):
    source = links
    driver.get(source)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser") 
    return soup

def listmake(word):
    num = isNone(word.select_one('.t_blue1 > sup'))
    china = isNone(word.select_one('dt> span'))
    poumsa =  isNone(word.select_one('dt'))
    content = isNone(word.select_one('font'))
    link = word.select_one('a').attrs['href']
    return num,china,poumsa,content,base_url +link

def isNone(word):
    if word != None and word.text != '':
        if len(word) > 3:
            index = startAndLast(re.sub('[\t\n\s]','',word.text))
            return re.sub('[\t\n\s]','',word.text)[index[0]:index[1]]
        return re.sub('[\t\n\s]','',word.text)
    else:
        return "Nope"

def startAndLast(word):
    ok = True
    start_index = 0
    last_index = 0
    for i in range(len(word)):
        if ok and word[i] == '「':
            start_index = i
        if word[i] == '」':
            last_index = i+1
            return start_index,last_index

search_data =  pyautogui.prompt("검색어를 입력해주세요.")

url = 'https://stdict.korean.go.kr/search/searchResult.do?pageSize=10&searchKeyword=' + str(search_data)

num = 0

search_soup = urlmake(url)

title = search_soup.select_one('.t_blue2')
if title != None:
    title = title.text.strip()
    words = search_soup.select('.result > li')
    for word in words:
        print(f'{columns[0]} : {title}')
        for i in range(len(listmake(word))):
            print(f'{columns[i+1]} : {listmake(word)[i]}') 
        print('----------------------------------\n')
        row = [title,*listmake(word)]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
else:
    text = search_soup.select_one('.no_result>li>div')
    if text == None:
        print(search_soup.select_one('.no_result').text)
    else:
        print(text.text)
        print('https://opendic.korean.go.kr/search/searchResult?query='+search_data)